In [1]:
!pip3 install pandas dash
!pip3 install jupyter_dash

In [2]:
#read the data
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash

In [3]:
airline_data =  pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetworkDV0101EN-SkillsNetwork/Data%20Files/airline_data.csv")
print(airline_data.columns)
data = airline_data[['CarrierDelay','WeatherDelay','LateAircraftDelay','NASDelay','SecurityDelay','Reporting_Airline','Flights','Month']]
data

HTTPError: HTTP Error 404: Not Found

In [ ]:
#create a dash application
app = JupyterDash(__name__)
app


In [ ]:
app.layout=html.Div(children=[html.H1('Flight delay time statistics',style={'textAlign':'center','color':'#503D36','font_size':30}),
                             html.Div(['Input Year:',dcc.Input(id='input-year',value='2010',type='number',style={'height':'35px','font_size':30}),],
                                     style={'font_size':30}),
                             html.Br(),
                             html.Br(),
                             html.Div([html.Div(dcc.Graph(id='carrier-plot')),
                                      html.Div(dcc.Graph(id='weather-plot'))],
                                      style={'display':'flex'}),
                             html.Div([html.Div(dcc.Graph(id='nas-plot')),
                                      html.Div(dcc.Graph(id='securityplot'))],
                                      style={'display':'flex'}),
                              html.Div(dcc.Graph(id='late-plot'),
                                       style={'width':'65%'}),
                                      ])
app.layout
#compute info function description
def compute_info(airline_data,entered_year):
    df=airline_data[airline_data['Year']==int(entered_year)]
    avg_car=df.groupby(['Month','ReportingAirline'])['CarrierDelay'].mean().reset_index()
    avg_weather=df.groupby(['Month','ReportingAirline'])['WeatherDelay'].mean().reset_index()
    avg_NAS=df.groupby(['Month','ReportingAirline'])['NASDelay'].mean().reset_index()
    avg_sec=df.groupby(['Month','ReportingAirline'])['SecurityDelay'].mean().reset_index()
    avg_late=df.groupby(['Month','ReportingAirline'])['LateAircraftDelay'].mean().reset_index()
#Callback function
@app.callback([
    Output(component_id='carrier-plot',component_property='figure'),
    Output(component_id='weather-plot',component_property='figure'),
    Output(component_id='nas-plot',component_property='figure'),
    Output(component_id='security-plot',component_property='figure'),
    Output(component_id='late-plot',component_property='figure'),],
    Input(component_id='input-year',component_property='value'))
#Computation to callback & return graph
def get_graph(entered_year):
    avg_car,avg_weather,avg_NAS,avg_sec,avg_late=compute_info(airline_data,entered_year)
    carrier_fig=px.line(avg_car,x='Month',y='CarrierDelay',color='ReportingAirline',title='Avg carrier delay time (min) by airline')
    weather_fig=px.line(avg_weather,x='Month',y='WeatherDelay',color='ReportingAirline',title='Avg weather delay time (min) by airline')
    nas_fig=px.line(avg_NAS,x='Month',y='NASDelay',color='ReportingAirline',title='Avg NAS delay time (min) by airline')
    security_fig=px.line(avg_sec,x='Month',y='SecurityDelay',color='ReportingAirline',title='Avg security delay time (min) by airline')
    late_fig=px.line(avg_late,x='Month',y='LateAircraftDelay',color='ReportingAirline',title='Avg late aircraft delay time (min) by airline')
    return[carrier_fig, weather_fig, nas_fig, security_fig,late_fig]
    

In [ ]:
if __name__ == '__main__':     app.run_server(mode='jupyterlab', port = 8090, dev_tools_ui=True, #debug=True,dev_tools_hot_reload =True, threaded=True)